<a href="https://colab.research.google.com/github/kylemath/DelCloseBot/blob/master/notebooks/DelCloseBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## DelCloseBot

Goal is to make a DelCloseBot using old youtube footage. Will have live actor filmed with webcam during show whose pose gets transformed into real time DelClose Poses shown to audience, with voice impressonation reading lines scripted from dialogue AI. 


*   Using https://github.com/datitran/face2face-demo
*   youtube video here https://www.youtube.com/watch?v=DjbQaSPPsqA
*   saved into .mp4 with https://www.onlinevideoconverter.com/youtube-converter 



Instructions:
* Connect colab to google drive 
* save the .mp4 video into a folder, 
* download the facial landmark model from http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
* after uncompressing locally and saving to google drive 

Tactics:
* Convert video into side by side images of video and face structure
* Train model on these side by side images to predict DelClose from Face pose
* Use live webam to get net face structure
* Convert new face structure to DelClose pose with model in real time

**Instal git repo**

In [1]:
!git clone https://github.com/datitran/face2face-demo.git --recursive
%cd face2face-demo


Cloning into 'face2face-demo'...
remote: Enumerating objects: 1, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 33 (delta 0), reused 0 (delta 0), pack-reused 32
Unpacking objects: 100% (33/33), done.
/content/face2face-demo


**Install anaconda and setup to use**

In [2]:
!wget -c https://repo.continuum.io/archive/Anaconda3-5.1.0-Linux-x86_64.sh
!chmod +x Anaconda3-5.1.0-Linux-x86_64.sh
!bash ./Anaconda3-5.1.0-Linux-x86_64.sh -b -f -p /usr/local

import sys
sys.path.append('/usr/local/lib/python3.6/site-packages/')


--2019-01-13 19:44:22--  https://repo.continuum.io/archive/Anaconda3-5.1.0-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.16.19.10, 104.16.18.10, 2606:4700::6810:120a, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.16.19.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 577996269 (551M) [application/x-sh]
Saving to: ‘Anaconda3-5.1.0-Linux-x86_64.sh’

Anaconda3-5.1.0-Lin 100%[===================>] 551.22M   116MB/s    in 5.1s    

2019-01-13 19:44:27 (109 MB/s) - ‘Anaconda3-5.1.0-Linux-x86_64.sh’ saved [577996269/577996269]

PREFIX=/usr/local
installing: python-3.6.4-hc3d631a_1 ...
Python 3.6.4 :: Anaconda, Inc.
installing: ca-certificates-2017.08.26-h1d4fec5_0 ...
installing: conda-env-2.6.0-h36134e3_1 ...
installing: intel-openmp-2018.0.0-hc7b2577_8 ...
installing: libgcc-ng-7.2.0-h7cc24e2_2 ...
installing: libgfortran-ng-7.2.0-h9f7466a_2 ...
installing: libstdcxx-ng-7.2.0-h7a57d05_2 ...
installing: bzip2-1.0.6-h9a117a8_4

** Create conda environment and activate it **

In [3]:
!conda env create -f environment.yml 
!source activate face2face-demo

Solving environment: - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - 

** Install some needed packages **

ISSUE - dlib install takes about 10 minutes to compile, find way to speed up?

In [4]:
!pip install opencv-python
!pip install imutils==0.4.3
!pip install tensorflow==1.2.1
!pip install dlib


    100% |████████████████████████████████| 25.4MB 46kB/s 
  Using cached https://files.pythonhosted.org/packages/a0/cf/5f19a892b73c1992b83fbe747556984e68b2c04a3967ab52ef093d89f929/imutils-0.4.3.tar.gz
  Running setup.py bdist_wheel for imutils ... - done
  Stored in directory: /root/.cache/pip/wheels/97/19/33/9c8b5e348f3e1d2d7f9a843ce736379a39fa6eb6c906287efe
Successfully built imutils
    100% |████████████████████████████████| 35.0MB 33kB/s 
  Using cached https://files.pythonhosted.org/packages/33/70/86c5fec937ea4964184d4d6c4f0b9551564f821e1c3575907639036d9b90/bleach-1.5.0-py2.py3-none-any.whl
    100% |████████████████████████████████| 92kB 8.6MB/s 
  Using cached https://files.pythonhosted.org/packages/6a/f7/ae34b6818b603e264f26fe7db2bd07850ce331ce2fde74b266d61f4a2d87/backports.weakref-1.0rc1-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/ae/ae/bcb60402c60932b32dfaf19bb53870b29eda2cd17551ba5639219fb5ebf9/html5lib-0.9999999.tar.gz
    100% |█████████████

**Mount your google drive to load and save from**


Note you have to follow the link here and get the activation code before proceeding

In [5]:
#mount google drive to load own images, follow link and get code and paste in below
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


** Processes the source video to find frames with faces, and pull out the face structure, save side by side image **

In [0]:
!python generate_train_data.py \
  --file /content/gdrive/My\ Drive/DeepLearning/DelCloseBot/DelCloseMono.mp4 \
  --num 400 \
  --landmark-model /content/gdrive/My\ Drive/DeepLearning/DelCloseBot/shape_predictor_68_face_landmarks.dat

In [8]:
# Clone the repo from Christopher Hesse's pix2pix TensorFlow implementation
%cd ..
!git clone https://github.com/affinelayer/pix2pix-tensorflow.git



/content
Cloning into 'pix2pix-tensorflow'...
remote: Enumerating objects: 261, done.
remote: Total 261 (delta 0), reused 0 (delta 0), pack-reused 261
Receiving objects: 100% (261/261), 13.33 MiB | 33.95 MiB/s, done.
Resolving deltas: 100% (105/105), done.


In [80]:
# Move the original and landmarks folder into the google drive folder
!cp -r face2face-demo/landmarks -r face2face-demo/original /content/gdrive/My\ Drive/DeepLearning/DelCloseBot/photos


/content/pix2pix-tensorflow


In [9]:
!ls

face2face-demo	gdrive	pix2pix-tensorflow  sample_data


In [10]:

# Go into the pix2pix-tensorflow folder
%cd pix2pix-tensorflow/


/content/pix2pix-tensorflow


In [0]:
# Resize original images
!python tools/process.py \
  --input_dir /content/gdrive/My\ Drive/DeepLearning/DelCloseBot/photos/original \
  --operation resize \
  --output_dir /content/gdrive/My\ Drive/DeepLearning/DelCloseBot/photos/original_resized
  
# Resize landmark images
!python tools/process.py \
  --input_dir /content/gdrive/My\ Drive/DeepLearning/DelCloseBot/photos/landmarks \
  --operation resize \
  --output_dir /content/gdrive/My\ Drive/DeepLearning/DelCloseBot/photos/landmarks_resized
  
# Combine both resized original and landmark images
!python tools/process.py \
  --input_dir /content/gdrive/My\ Drive/DeepLearning/DelCloseBot/photos/landmarks_resized \
  --b_dir /content/gdrive/My\ Drive/DeepLearning/DelCloseBot/photos/original_resized \
  --operation combine \
  --output_dir /content/gdrive/My\ Drive/DeepLearning/DelCloseBot/photos/combined
  
# Split into train/val set
!python tools/split.py \
  --dir /content/gdrive/My\ Drive/DeepLearning/DelCloseBot/photos/combined

In [11]:
# Train the model on the data for the first time
!python pix2pix.py \
  --mode train \
  --output_dir /content/gdrive/My\ Drive/DeepLearning/DelCloseBot/face2face-model \
  --max_epochs 200 \
  --input_dir /content/gdrive/My\ Drive/DeepLearning/DelCloseBot/photos/combined/train \
  --which_direction AtoB \
  --display_freq 50 \
  --save_freq 200

aspect_ratio = 1.0
batch_size = 1
beta1 = 0.5
checkpoint = None
display_freq = 50
flip = True
gan_weight = 1.0
input_dir = /content/gdrive/My Drive/DeepLearning/DelCloseBot/photos/combined/train
l1_weight = 100.0
lab_colorization = False
lr = 0.0002
max_epochs = 200
max_steps = None
mode = train
ndf = 64
ngf = 64
output_dir = /content/gdrive/My Drive/DeepLearning/DelCloseBot/face2face-model
output_filetype = png
progress_freq = 50
save_freq = 200
scale_size = 286
seed = 472535221
separable_conv = False
summary_freq = 100
trace_freq = 0
which_direction = AtoB
examples count = 320
2019-01-13 20:10:18.582127: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.1 instructions, but these are available on your machine and could speed up CPU computations.
2019-01-13 20:10:18.582186: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.2 instructions, but these are available on your machine and

In [0]:
# Train the model on the data from a checkpoint
!python pix2pix-tensorflow/pix2pix.py \
  --mode train \
  --output_dir /content/gdrive/My\ Drive/DeepLearning/DelCloseBot/face2face-model \
  --max_epochs 200 \
  --input_dir /content/gdrive/My\ Drive/DeepLearning/DelCloseBot/photos/combined/train \
  --which_direction AtoB \
  --display_freq 50 \
  --checkpoint /content/gdrive/My\ Drive/DeepLearning/DelCloseBot/face2face-model \
  --save_freq 200

aspect_ratio = 1.0
batch_size = 1
beta1 = 0.5
checkpoint = /content/gdrive/My Drive/DeepLearning/DelCloseBot/face2face-model
display_freq = 50
flip = True
gan_weight = 1.0
input_dir = /content/gdrive/My Drive/DeepLearning/DelCloseBot/photos/combined/train
l1_weight = 100.0
lab_colorization = False
lr = 0.0002
max_epochs = 200
max_steps = None
mode = train
ndf = 64
ngf = 64
output_dir = /content/gdrive/My Drive/DeepLearning/DelCloseBot/face2face-model
output_filetype = png
progress_freq = 50
save_freq = 200
scale_size = 286
seed = 699686393
separable_conv = False
summary_freq = 100
trace_freq = 0
which_direction = AtoB
examples count = 320
2019-01-14 03:18:18.719988: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.1 instructions, but these are available on your machine and could speed up CPU computations.
2019-01-14 03:18:18.720049: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE

In [27]:
%pwd
!ls

face2face-demo	gdrive	pix2pix-tensorflow  sample_data


Reduce the model

In [28]:
!python face2face-demo/reduce_model.py \
  --model-input gdrive/My\ Drive/DeepLearning/DelCloseBot/face2face-model \
  --model-output gdrive/My\ Drive/DeepLearning/DelCloseBot/face2face-model-reduced

2019-01-14 03:17:54.288856: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.1 instructions, but these are available on your machine and could speed up CPU computations.
2019-01-14 03:17:54.288921: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.2 instructions, but these are available on your machine and could speed up CPU computations.
2019-01-14 03:17:54.288939: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use AVX instructions, but these are available on your machine and could speed up CPU computations.
2019-01-14 03:17:54.288963: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use AVX2 instructions, but these are available on your machine and could speed up CPU computations.
2019-01-14 03:17:54.288987: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't

Freeze the reduced model to single file

In [29]:
!python face2face-demo/freeze_model.py --model-folder gdrive/My\ Drive/DeepLearning/DelCloseBot/face2face-model-reduced

2019-01-14 03:18:02.950428: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.1 instructions, but these are available on your machine and could speed up CPU computations.
2019-01-14 03:18:02.950488: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.2 instructions, but these are available on your machine and could speed up CPU computations.
2019-01-14 03:18:02.950505: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use AVX instructions, but these are available on your machine and could speed up CPU computations.
2019-01-14 03:18:02.950522: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use AVX2 instructions, but these are available on your machine and could speed up CPU computations.
2019-01-14 03:18:02.950535: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't

Now run the demo - may need to download model and do offline

download frozen model
https://drive.google.com/open?id=10hIHuvVDsOVzt2YgZzXW0fsstVTeodG3

open terminal








In [0]:
## first time

git clone https://github.com/kylemath/face2face-demo.git

cd face2face-demo

conda env create -f environment.yml

source activate face2face-demo

# show 0 to show the live webcam too, show 2 to show large output image only, show 1 is face tracking
python run_webcam_only.py \
  --source 0 \
  --show 2 \
  --landmark-model /Users/kylemathewson/face2face-demo/shape_predictor_68_face_landmarks.dat \
  --tf-model /Users/kylemathewson/face2face-demo/frozen_model.pb


## subsequent times

cd face2face-demo
source activate face2face-demo

# show 0 to show the live webcam too, show 2 to show large output image only, show 1 is face tracking
python run_webcam_only.py \
  --source 0 \
  --show 2 \
  --landmark-model /Users/kylemathewson/face2face-demo/shape_predictor_68_face_landmarks.dat \
  --tf-model /Users/kylemathewson/face2face-demo/frozen_model.pb